# Színváltó LED parancsra

Az egyik barátodban felmerült, hogy ő bizony vadászpilóta szeretne lenni. Rákerestetek a neten, hogy milyen feltételeknek kell megfelelni ehhez és többek között kiderült, hogy a jelentkező nem lehet színvak. Eldöntöttétek, hogy ezt a feltételt bizony leellenőrzitek. 

Tesztelni szeretnéd a barátod színfelismerő képességét. Kitaláltad, hogy készítesz egy szerkezetet, aminek begépeled, hogy milyen színt szeretnél, hogy mutasson, anélkül, hogy a barátod látná, mit gépelsz. Miután megjelent a kiválasztott szín, megkéred a barátod, hogy találja ki a nevét. Evvel a módszerrel gyorsan letesztelhetitek, hogy megfeleltek-e a vadászpilóták egyik követelményének.

## Mit fogsz készíteni?

Egy RGB LED-ből és három ellenállásból álló áramkört fogunk készíteni. Ehhez írunk kódot ami a begépelt szöveg alapján változtatja a színét.

## Mit tanulsz meg?

Az RGB LED áramkör elkészítésével a következőket tanulod meg:

* Hogyan tudsz egy egyszerű áramkört összerakni egy breadboard, RGB LED, ellenállások és drótok segítségével.
* Hogyan programozd be a Raspberry Pi GPIO kimeneteleit a **gpiozero** modullal.
* Hogyan használjuk az *RGBLED* objektumokat és változtasd azoknak a színét.
* Hogyan használd a *dictionary* Python típust összefüggések tárolására.

## A projekt részletekre bontása

* Inicializálni az RGB LED objektumot
* Színkód könyvtár elkészítése
* Begépelt szín nevéhez RGB kódot rendelni
* Végtelen ciklusban kérdezni a LED színét és azt átváltani.

## Áramköri elemek listája

a) [Raspberry PI](https://malnapc.hu/yis/raspberry-pi/rpi-panelek) 

b) [Jumper wires female/male](https://www.ret.hu/shop/product/e-call/jumper-vezetek-szet_53-22-63) 

c) RGB LED, $I_{max}$ = 20 mA-es: [itt vásárolhatsz](https://www.tme.eu/hu/details/osta56a1a-c/tht-led-diodak-egyeb/optosupply/)

d) 3db 220 - 560 Ohm közötti [ellenállás](https://www.tme.eu/hu/katalog/tht-metal-film-ellenallasok-0-6w_100289/?s_order=desc&search=ellenallas&s_field=1000011)

e) 1db [Breadboard](https://www.tme.eu/hu/katalog/muhelyfelszereles_112607/?s_order=desc&search=breadboard&s_field=1000011)

## A kapcsolási rajz

<img src="prog04_circuit.png" width=600 height=400 />

A fenti ábrához hasonlóan kapcsoljuk össze az áramköri elemeket és a Raspberry Pi-t.

1) Helyezzük az RGB LED lábait különboző sorokba. **FONTOS**: Ha közös katódú (-) LED-ünk van, akkor a leghosszabb lábat, a közös katódot, a földelésre, **GND** kell kapcsolni, ha közös anódú (+) LED-ünk van, akkor a leghosszabb lábat a 3.3V-os (vagy 5V-os) feszültségre kell kapcsolni. A rajzon mi a közös katódú LED-et ábrázoljuk.

A LED anódjának és katódjának megállapítására használhatunk egy multimétert. Csatlakoztasd a fekete mérőzsinórt a COM (föld) és a piros mérőzsinórt a VΩmA jelzésű hüvelybe. Forgasd el a méréshatárváltó kapcsolót a folytonosság mérés állapotba. Érintsd a két zsínórt a leghosszabb lábhoz és egy másikhoz és ha a LED halványan pislákol, akkor az a láb amelyikhez a fekete zsinórt érintetted a katód, a másik az anód. Ha nem pislákol akkor cseréld meg a zsinórok és a lábak érintkezését, hogy világítson.

<img src="rgb-led-pin.png" width=500 height=400 />

RGB LED sematikus ábrázolása. A (-) és (+) jelölés (leghosszabb láb) a közös katódot vagy anódot jelentik, míg az R, G és B a színek kivezetését képviselik.

2) Az RGB LED leghosszabb lábát (közös katód) kössük össze a Raspberry Pi egyik **GND** jelölésű tüskéjével egy jumper drót segítségével.

3) A többi láb sorába az ábrának megfelelően kössük be az ellenállásokat.

4) A piros (R) lábhoz tartozó ellenállás sorát kössük a **17**-es GPIO tüskéhez, a zöldet (G) a **27**-eshez és a kéket a **22**-hez.

## A kód
Nyissunk meg egy új python fájlt és mentsük el pl. ```led_RGB_text.py``` név alatt.

### A LED-ek kipróbálása

Miután elkészítettük az áramkört, meg kell írnunk a kódot ami utasítja a Raspberry Pi-t, hogyan viselkedjen az RGB LED.

Először betöltjük a ```gpiozero``` modulból az ```RGBLED``` objektumot ami lehetővé teszik, hogy a Raspi kommunikáljon a LED-del a **GPIO** tüskéken keresztül. Ez mellett beolvassuk még a ```sleep``` függvényt is, amit a python várakoztatására használunk majd.

```led_RGB_text.py```:

In [ ]:
from gpiozero import RGBLED
from time import sleep

Az objektum beolvasása után inicializálunk egy ```RGBLED``` objektumot, amit ```led```-nek nevezünk el. Az argumentumok között azt is megadjuk, hogy melyik színt melyik GPIO tüskére kapcsoltuk. Emellett érdemes megemlíteni az ```active_high``` argumentumot, mert annak tükrében, hogy közös katódú vagy anódú LED-et használunk, ```True```-nak vagy ```False```-nak kell lennie. A mi pépldánkban közös katódú LED szerepel, ezért ```active_high=True```.

```led_RGB_text.py```:

In [ ]:
from gpiozero import LED
from time import sleep

led = RGBLED(red=17, green=27, blue=22, active_high = True)

Az inicializálás után már tesztelhetjük is a LED-et, hogy működik-e. Az alap 3 színre van metódus, azaz a ```led.red```, ```led.green``` és ```led.blue``` paranccsokkal hivatkozhatunk rájuk. Ha 0-át rendelünk hozzájuk, akkor nem világítanak, míg ha 1-et akkor a max intenzitásukkal világítanak. Az intenzitás mértékét 0 és 1 között lehet variálni. Pl. a piros LED-et a következőképpen is szabályozhatjuk:

```led_RGB_text.py```:

In [ ]:
from gpiozero import RGBLED
from time import sleep


led = RGBLED(red=17, green=27, blue=22)

led.red = 1  # full red
sleep(1)
led.red = 0.5  # half red
sleep(1)

Az ```RGBLED``` inicializálásakor van még egy érdekes argumentum, a ```pwm=True```. A pwm a *Pulsed Width Modulation* rövidítése és lényegében a GPIO tüske által leadott effektív teljesítmény szabályozására szolgál. A digitális jelek (mint a GPIO tüske jelei) vagy 0V, alacsony, vagy pl. 3.3V, magas állapotban vannak. Nem tudnak köztes állapotot felvenni, mint az analóg jelek. Ha szabályozni akarjuk pl. a LED fényerejét, akkor ki-be kell kapcsolni a digitális jelet, azaz modulálni időben. Ezt teszi lehetővé a már előbb említett ```pwm=True``` argumentum. Ha hamisra állítanánk, akkor nem tudnánk a színek intenzitását szabályozni.   

### Színnév és színkód a könyvtárban

A következő lépésben a szín nevének és a hozzá tartozó RGB kódnak az összekötésén dolgzunk a Python *dictionary* típusának segítségével. ```{}``` jellel definiáljuk a dictionary-ket. Definiáláskor meg kell adni egy kulcsot, stringként és a hozzá tartozó értéket, pl. ```d = {'name': 'George', 'age': 20}```. Több kulcsot is megadhatunk, illetve a hozzájuk tartozó értékek is lehetnek különböző típusúak.

Most a kódunkat kiegészítjük egy színnév kulcsú és színkód értékű dictionary-t, ```colors_dict```. Majd kipróbálunk néhány metódust amivel kezelni lehet a dictionary-ket.

```led_RGB_text.py```:

In [ ]:
from gpiozero import RGBLED
from time import sleep

led = RGBLED(red=17, green=27, blue=22, active_high = True)

colors_dict = {'red': (1,0,0),
          'green': (0,1,0),
          'blue': (0,0,1),
          'white': (1,1,1),
          'black': (0,0,0),
          'yellow': (1,1,0),
          'cyan': (0,1,1),
          'magenta': (1,0,1),
          'silver': (192/255,192/255,192/255),
          'gray': (128/255,128/255,128/255),
          'deep pink': (255/255,20/255,147/255),
}

print(colors_dict['red'])   # kiírja a 'red' kulcshoz tartozó értéket
print(colors_dict['white']) # kiírja a 'white' kulcshoz tartozó értéket
print(colors_dict.keys())   # kiírja a dictionary kulcsait
print(colors_dict.values()) # kiírja a dictionary összes értékét
print(colors_dict.get('white', 0))  # kiírja a 'white'-oz tartozó értéket, 
                                    # ha nem lenne 'white' kulcs, akkor 0-t írna ki
print(colors_dict.get('orange', 'No match')) # mivel nincs 'orange' kulcs, a kinyomtatott
                                             # érték a 'No match'

Mentsük el a kódot és futtassuk le.

### Függvény ami a színnév alapján visszaadja a színkódot

A kódunkat most kiegészítjük egy függvénnyel, ```set_color``` ami a bemenő színnév alapján visszaadja a hozzá tartozó színkódot, amit a második paraméterként megadott, ```cm = colors_dict```, dictionary-ben keres meg. Amennyiben, a beírt színnév nincs a kulcsok között, ```None``` értéket fog visszaadni. 

Néhány példán keresztül próbáljuk ki a működését. A már említett második paramétert nem kötelező definiálnunk, hiszen van neki egy alapértelmezett értéke, ami a ```colors_dict```. Ha másmilyen szín definíciókkal szeretnénk dolgozni, akkor a ```cm``` paraméter helyére egy új dictionary-t kell megadni a megfelelő kulcs és érték párokkal. 

```led_RGB_text.py```:

In [ ]:
from gpiozero import RGBLED
from time import sleep

led = RGBLED(red=17, green=27, blue=22, active_high = True)

colors_dict = {'red': (1,0,0),
          'green': (0,1,0),
          'blue': (0,0,1),
          'white': (1,1,1),
          'black': (0,0,0),
          'yellow': (1,1,0),
          'cyan': (0,1,1),
          'magenta': (1,0,1),
          'silver': (192/255,192/255,192/255),
          'gray': (128/255,128/255,128/255),
          'deep pink': (255/255,20/255,147/255),
}

def set_color(color, cm = colors_dict):
    col = colors_dict.get(color, None)
    return col

print(set_color('green', cm = colors_dict))  # (0,1,0)
print(set_color('cyan'))                     # (0,1,1)
print(set_color('orange'))                   # None

### Gépeléssel kontrollálni a LED színét

Nem maradt más, mint elindítani egy végtelen ```while``` ciklust, aminek minden ciklusában megkérdezzük milyen színt állítson be a Raspberry Pi.

Pythonban az ```input``` parancsot használjuk arra, hogy beolvassunk a terminálból stringeket és elmentsük azt egy változóba, ```answer```. Ebben az esetben válaszként egy szín nevet várunk. Ezt a választ bemenő paraméterként megadjuk a ```set_color``` függvénynek, ami visszaadja a szín RGB kódját. 

Amennyiben olyan színt adtunk a ```set_color``` függvénynek, ami nincs a ```colors_dict``` dictionary kulcsai között akkor ```None```-t kapunk vissza. Ezt kivizsgálva az ```if color:``` szerkezetben az ```else:``` szerkezet aktiválódik, ami azt mondja, hogy folytassuk a ciklust, így ismét felteszi a kérdést, hogy milyen színt szeretnénk látni. Ha létező színt választunk, akkor az ```if color:``` szerekezetben beállítjuk a ```led``` színeit az általunk kiválasztottra. 

In [ ]:
from gpiozero import RGBLED
from time import sleep

led = RGBLED(red=17, green=27, blue=22, active_high = True)

colors_dict = {'red': (1,0,0),
          'green': (0,1,0),
          'blue': (0,0,1),
          'white': (1,1,1),
          'black': (0,0,0),
          'yellow': (1,1,0),
          'cyan': (0,1,1),
          'magenta': (1,0,1),
          'silver': (192/255,192/255,192/255),
          'gray': (128/255,128/255,128/255),
          'deep pink': (255/255,20/255,147/255),
}

def set_color(color, cm = colors_dict):
    col = colors_dict.get(color, None)
    return col

while True:
    answer = input('What color do you want to see? ')
    color = set_color(answer)
    if color:
        led.color = color
    else:
        continue

Mentsük el a kódot, majd teszteljük le a működését.

## A projekt tesztelése

Miután összeszereltük az áramkört és a kódot is megírtuk, amit pl. ```led_RGB_text.py``` név alatt mentettünk el, megnyithatunk a Raspberry Pi operációs rendszerén egy terminált. A terminálban a ```cd 'mappa név'``` paranccsal elnavigálunk abba a mappába, ahova a ```led_RGB_text.py```-t elmentettük. Ott begépelve a ```python led_RGB_text.py``` parancsot, letesztelhetjük a programunk működését. Ha minden jól megy akkor az RGB LED különböző színekben fog világítani annak függvényében, hogy milyen színt gépelünk be.

Hibaüzenetek esetén ki kell deríteni mi lehetett a probléma, pl. elgépelés, egy modul hiányzik, sorok megfelelő behúzása, idézőjel lemaradása stb. A hibaüzenet legtöbbször segít abban, hogy melyik sorban találta a hibát és hogy mi volt az. Egy kis gyakorlással bele lehet jönni azok értelmezésébe, valamint interneten is rá lehet keresni a hibaüzenet jelentésére és annak lehetséges elhárítására.

## Mit lehet javítani/továbbfejleszteni?

* Megjeleníteni egy színt a LED-en, majd a felhasználónak be kell gépelni melyik szín az. Eldünteni jót írt-e vagy sem.
* Gombnyomásra random színeket megjeleníteni és begépelni azok színét, majd eldönteni, helyes volt-e a tipp.

Írd meg kommentben, hogy szerinted mivel lehetne még feldobni ezt a kis programot!

## Referencia

1) gpiozero leírás - [https://gpiozero.readthedocs.io/en/stable/recipes.html](https://gpiozero.readthedocs.io/en/stable/recipes.html)

2) LED objektum leírása - [https://gpiozero.readthedocs.io/en/stable/api_output.html#led](https://gpiozero.readthedocs.io/en/stable/api_output.html#led)

3) RGBLED objektum leírása - [https://gpiozero.readthedocs.io/en/stable/api_output.html#rgbled](https://gpiozero.readthedocs.io/en/stable/api_output.html#rgbled)

4) A *dictionary* típus leírása - [https://www.learnpython.org/en/Dictionaries](https://www.learnpython.org/en/Dictionaries)